# [MLP_enhance] Fast Baseline Model

이 노트북은 **속도와 효율성**을 중시하는 **Fast Baseline** 모델 구축을 목표로 합니다.  
복잡한 앙상블이나 무거운 학습 과정을 배제하고, **단일 모델**로 빠르게 실험 결과를 확인합니다.

### 📌 핵심 전략 (Key Strategy)
1.  **Clean Data Pipeline**: `anchors` 테이블 병합 과정을 제거하고, 정제된 `features_ml_clean`과 `labels`만을 사용하여 데이터 무결성을 확보했습니다.
2.  **Use Raw Distribution**: **SMOTE(오버샘플링)**를 제거하고 원본 데이터 분포 그대로 학습하여, 실제 환경과 가장 유사한 조건에서 성능을 평가합니다.
3.  **Fast Hyperparameters**: Epochs를 10회로 제한하고, 모델 크기를 최적화(Hidden 128)하여 실행 속도를 극대화했습니다.

---

## 1. 라이브러리 임포트 및 설정

- **시스템 경로**: 프로젝트 Root를 추가하여 커스텀 모듈 호출을 가능하게 합니다.
- **라이브러리**: PyTorch, Pandas, Numpy 등 필수 패키지를 로드합니다.
- **가속기 확인**: Mac M1/M2/M3(MPS) 또는 CUDA 사용 가능 여부를 확인합니다.
- **모델 로드**: `models/model_definitions.py`에서 `MLP_enhance` 클래스를 가져옵니다.

In [ ]:
import sys
import os
import pathlib

# Root 경로 잡기
sys.path.append(os.path.abspath("../../"))

import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd
import json
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import StandardScaler
from IPython.display import display, Markdown

# [MPS Acceleration] 가속 지원 확인
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
print(f"🚀 Using Device: {device}")

# [Model Import] MLP_enhance 모델 로드
from models.model_definitions import MLP_enhance
print("✅ MLP_enhance Model Imported.")


## 2. 데이터 로드 및 전처리 (Data Loading)

- **Parquet 로드**: Feature와 Label 데이터를 각각 불러옵니다.
- **Type Casting**: `user_id` 매칭 오류 방지를 위해 문자열로 변환합니다.
- **Inner Join**: Feature와 Label이 모두 존재하는 유효한 샘플만 병합합니다. (`anchors` 병합 제거됨)

In [ ]:
print('📂 Loading Data...')
base_path = "/Users/gimdabin/SKN23-2nd-3Team/data/processed/"

# Features & Labels 로드
features = pd.read_parquet(base_path + "features_ml_clean.parquet")
labels = pd.read_parquet(base_path + "labels.parquet")

# ID 컬럼 타입 통일
features['user_id'] = features['user_id'].astype(str)
labels['user_id'] = labels['user_id'].astype(str)

# 데이터 병합 (Inner Join)
data = features.merge(labels, on=['user_id', 'anchor_time'], how='inner')
data['target'] = (data['label'] == 'm2').astype(int)

# X(Features)와 y(Target) 분리
feature_cols = [c for c in features.columns if c not in ['user_id', 'anchor_time']]
X = data[feature_cols].copy().fillna(0)
y = data['target'].values

print(f"✅ Data Concatenation Complete. Shape: {data.shape}")


## 3. 데이터 분할 (Time-based Split)

- **Time-based Split**: 미래 데이터 참조(Leakage)를 막기 위해 시간 순서가 적용된 `split` 컬럼을 사용합니다.
- **데이터셋 구분**: Train(학습), Val(검증), Test(최종 평가)로 나눕니다.
- **분포 확인**: 각 Set별 타겟 비율(Base Rate)을 출력하여 데이터 치우침을 확인합니다.

In [ ]:
assert "split" in data.columns, "Missing 'split' column!"

split = data["split"].astype(str).str.lower().values
train_mask = split == "train"
val_mask   = np.isin(split, ["val", "valid", "validation"])
test_mask  = split == "test"

X_train, y_train = X.loc[train_mask].values, y[train_mask]
X_val, y_val     = X.loc[val_mask].values, y[val_mask]
X_test, y_test   = X.loc[test_mask].values, y[test_mask]

print(f"🔹 Train Set: {len(X_train)} samples")
print(f"🔹 Val   Set: {len(X_val)} samples")
print(f"🔹 Test  Set: {len(X_test)} samples")

print(f"📊 Train Base Rate: {y_train.mean():.4f}")
print(f"📊 Val   Base Rate: {y_val.mean():.4f}")
print(f"📊 Test  Base Rate: {y_test.mean():.4f}")


## 4. 데이터 스케일링 및 로더 생성 (Scaling & Loader)

- **StandardScaler**: 평균 0, 분산 1로 정규화합니다. `fit`은 오직 **Train Set**에만 적용합니다.
- **No SMOTE**: 인위적인 오버샘플링을 제거하고, 실제 데이터 분포(Raw Distribution)를 그대로 학습합니다.
- **DataLoader**: 배치(Batch) 단위 학습을 위해 PyTorch DataLoader를 생성합니다.

In [ ]:
# Scaling
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

# DataLoader Setup
BATCH_SIZE = 256
train_loader = DataLoader(TensorDataset(torch.FloatTensor(X_train_scaled), torch.FloatTensor(y_train)), batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(TensorDataset(torch.FloatTensor(X_val_scaled), torch.FloatTensor(y_val)), batch_size=BATCH_SIZE)
test_loader = DataLoader(TensorDataset(torch.FloatTensor(X_test_scaled), torch.FloatTensor(y_test)), batch_size=BATCH_SIZE)

print("✅ Preprocessing Complete (No SMOTE, Raw Distribution).")


## 5. 하이퍼파라미터 설정 (Fast Config)

빠른 실험과 Baseline 수립을 위해 경량화된 파라미터를 사용합니다.

- **Epochs**: 10 (충분한 수렴 확인 가능)
- **Hidden Dim**: 128 (모델 복잡도 최적화)
- **Optimizer**: Adam (안정적인 수렴)

In [ ]:
best_params = {
    'lr': 0.001,
    'weight_decay': 0.00001,
    'hidden_dim': 128,
    'dropout_rate': 0.3,
    'epochs': 10,
    'activation': 'relu',
    'optimizer': 'Adam'
}
print(f"⚙️ Applied Fast Hyperparameters: {best_params}")


## 6. 모델 학습 (Model Training)

- **Training Loop**: 설정된 Epoch만큼 반복하며 모델을 업데이트합니다.
- **Loss Function**: 이진 분류를 위해 `BCEWithLogitsLoss`를 사용합니다.
- **Monitoring**: 매 Epoch마다 Train Loss와 Val Loss를 출력하여 과적합 여부를 체크합니다.

In [ ]:
model = MLP_enhance(
    X.shape[1], 
    hidden_dim=best_params['hidden_dim'], 
    dropout_rate=best_params['dropout_rate'], 
    activation=best_params['activation']
).to(device)

criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=best_params['lr'], weight_decay=best_params['weight_decay'])

print("🚀 Starting Training (Fast Mode)...")

for epoch in range(best_params['epochs']):
    model.train()
    epoch_loss = 0
    for inputs, targets in train_loader:
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = model(inputs).squeeze()
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
    
    # Validation Loss Check
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for inputs, targets in val_loader:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs).squeeze()
            val_loss += criterion(outputs, targets).item()
    
    print(f'Epoch {epoch+1:02d}/{best_params["epochs"]}: Train Loss {epoch_loss/len(train_loader):.4f}, Val Loss {val_loss/len(val_loader):.4f}')


## 7. 모델 평가 (Evaluation)

- **Test Set Evaluation**: 과적합되지 않은 순수한 Test 데이터로 최종 성능을 측정합니다.
- **Metrics**: ROC-AUC, PR-AUC, Precision, Recall 등 주요 지표를 계산합니다.
- **Ranking**: 상위 n% 유저에 대한 리프트(Lift) 성능을 확인하여 마케팅 효용성을 검증합니다.

In [ ]:
from app.utils.metrics import evaluate_churn_metrics

print("Evaluating on Test Set...")
model.eval()
all_probs = []
all_true = []

with torch.no_grad():
    for inputs, targets in test_loader:
        inputs = inputs.to(device)
        outputs = model(inputs).squeeze()
        probs = torch.sigmoid(outputs)
        all_probs.extend(probs.cpu().numpy())
        all_true.extend(targets.numpy())

metrics_result = evaluate_churn_metrics(
    y_true=np.array(all_true),
    y_prob=np.array(all_probs)
)

# Ranking 분리
if 'ranking' in metrics_result:
    ranking_list = metrics_result.pop('ranking')
    top_k_df = pd.DataFrame(ranking_list)
else:
    top_k_df = pd.DataFrame()

metrics = metrics_result

display(Markdown("### 📊 Fast Baseline Performance"))
display(pd.DataFrame(list(metrics.items()), columns=['KPI', 'Value']))
display(Markdown("### 📈 Top K% Ranking"))
display(top_k_df)


## 8. 결과 저장 (Artifact Saving)

- **JSON Export**: 모델 평가 결과와 메타데이터를 표준화된 JSON 포맷으로 저장합니다.
- **Path**: `models/eval/dlmlp_enhance/`

In [ ]:
import json
import pathlib

MODEL_ID = "dlmlp_enhance"
EVAL_DIR = pathlib.Path(f"../../models/eval/{MODEL_ID}")
EVAL_DIR.mkdir(parents=True, exist_ok=True)

# Model Card
model_card = {
    "model_id": MODEL_ID,
    "display_name": "MLP Enhance (Fast Baseline)",
    "category": "DL",
    "split": "test"
}
with open(EVAL_DIR / "model_card.json", "w") as f:
    json.dump(model_card, f, indent=2, ensure_ascii=False)

# PR Metrics
pr_metrics = {
    "model_id": MODEL_ID,
    "split": "test",
    "pr_auc": float(metrics.get("PR-AUC (Average Precision)", 0.0))
}
with open(EVAL_DIR / "pr_metrics.json", "w") as f:
    json.dump(pr_metrics, f, indent=2, ensure_ascii=False)

# Top K Metrics
current_base_rate = float(np.mean(all_true)) if 'all_true' in locals() else 0.0
metrics_by_k = []
if not top_k_df.empty:
    for _, row in top_k_df.iterrows():
        k_str = str(row.get('Top_K', '0')).replace('%', '')
        try: k_val = int(k_str)
        except: k_val = 0
        metrics_by_k.append({
            "k_pct": k_val,
            "precision_at_k": float(row.get('Precision', 0.0)),
            "recall_at_k": float(row.get('Recall', 0.0)),
            "lift_at_k": float(row.get('Lift', 0.0))
        })

topk_output = {
    "model_id": MODEL_ID,
    "split": "test",
    "base_rate": current_base_rate,
    "metrics_by_k": metrics_by_k
}
with open(EVAL_DIR / "topk_metrics.json", "w") as f:
    json.dump(topk_output, f, indent=2, ensure_ascii=False)

print(f"✅ All artifacts saved to {EVAL_DIR} (LG Style Format)")
